# Spatial Data Analysis

This notebook provides an introduction to spatial data analysis and geospatial data processing using Python.

## Topics Covered:
1. Introduction to Spatial Data
2. Working with Geospatial Libraries
3. Reading and Visualizing Geographic Data
4. Spatial Operations and Analysis
5. Coordinate Reference Systems
6. Spatial Joins and Relationships
7. Distance and Buffer Analysis
8. Spatial Clustering

## 1. Setup and Imports

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon, MultiPoint
from shapely import wkt
import folium
from folium import plugins

# Set visualization parameters
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

## 2. Creating Geometric Objects

Let's start by creating basic geometric shapes using Shapely.

In [ ]:
# Create points
point1 = Point(0, 0)
point2 = Point(1, 1)
point3 = Point(2, 2)

print(f"Point 1: {point1}")
print(f"Point 1 coordinates: ({point1.x}, {point1.y})")

# Create a line
line = LineString([(0, 0), (1, 1), (2, 1), (3, 0)])
print(f"\nLine: {line}")
print(f"Line length: {line.length:.2f}")

# Create a polygon
polygon = Polygon([(0, 0), (2, 0), (2, 2), (0, 2), (0, 0)])
print(f"\nPolygon: {polygon}")
print(f"Polygon area: {polygon.area:.2f}")
print(f"Polygon perimeter: {polygon.length:.2f}")

## 3. Visualizing Geometric Objects

In [ ]:
# Visualize different geometric objects
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot points
points = MultiPoint([point1, point2, point3])
x_coords = [p.x for p in points.geoms]
y_coords = [p.y for p in points.geoms]
axes[0].scatter(x_coords, y_coords, c='red', s=100, zorder=5)
axes[0].set_title('Points')
axes[0].grid(True, alpha=0.3)
axes[0].set_aspect('equal')

# Plot line
x, y = line.xy
axes[1].plot(x, y, 'b-', linewidth=2)
axes[1].scatter(x, y, c='red', s=50, zorder=5)
axes[1].set_title('LineString')
axes[1].grid(True, alpha=0.3)
axes[1].set_aspect('equal')

# Plot polygon
x, y = polygon.exterior.xy
axes[2].fill(x, y, alpha=0.5, fc='lightblue', ec='blue', linewidth=2)
axes[2].set_title('Polygon')
axes[2].grid(True, alpha=0.3)
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

## 4. Creating a GeoDataFrame

GeoDataFrame is the core data structure in GeoPandas, extending pandas DataFrame with geospatial capabilities.

In [ ]:
# Create sample data for cities
cities_data = {
    'City': ['Moscow', 'Saint Petersburg', 'Novosibirsk', 'Yekaterinburg', 'Kazan'],
    'Population': [12500000, 5400000, 1600000, 1500000, 1250000],
    'Latitude': [55.7558, 59.9311, 55.0084, 56.8389, 55.7964],
    'Longitude': [37.6173, 30.3609, 82.9357, 60.6057, 49.1089]
}

# Create a DataFrame
df = pd.DataFrame(cities_data)

# Convert to GeoDataFrame
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf_cities = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

print(gdf_cities.head())
print(f"\nCRS: {gdf_cities.crs}")

## 5. Visualizing GeoDataFrame

In [ ]:
# Plot cities with population-based sizing
fig, ax = plt.subplots(figsize=(12, 8))

gdf_cities.plot(ax=ax, 
                markersize=gdf_cities['Population']/50000,
                color='red', 
                alpha=0.6,
                edgecolor='black',
                linewidth=1)

# Add city labels
for idx, row in gdf_cities.iterrows():
    ax.annotate(text=row['City'], 
                xy=(row['Longitude'], row['Latitude']),
                xytext=(5, 5), 
                textcoords='offset points',
                fontsize=10,
                fontweight='bold')

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Major Russian Cities (Size proportional to population)')
ax.grid(True, alpha=0.3)
plt.show()

## 6. Working with Built-in Datasets

In [ ]:
# Load built-in world dataset
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

print(f"Shape: {world.shape}")
print(f"\nColumns: {world.columns.tolist()}")
print(f"\nFirst few rows:")
print(world.head())

In [ ]:
# Visualize world map
fig, ax = plt.subplots(figsize=(15, 10))

world.plot(ax=ax, 
           color='lightblue', 
           edgecolor='black', 
           linewidth=0.5)

ax.set_title('World Map', fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.grid(True, alpha=0.3)
plt.show()

## 7. Spatial Filtering and Selection

In [ ]:
# Filter countries by continent
europe = world[world['continent'] == 'Europe']
asia = world[world['continent'] == 'Asia']

print(f"Number of European countries: {len(europe)}")
print(f"Number of Asian countries: {len(asia)}")

# Visualize Europe
fig, ax = plt.subplots(figsize=(12, 10))
europe.plot(ax=ax, 
            column='pop_est', 
            cmap='YlOrRd', 
            legend=True,
            edgecolor='black',
            linewidth=0.5,
            legend_kwds={'label': 'Population'})

ax.set_title('European Countries by Population', fontsize=14, fontweight='bold')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.show()

## 8. Spatial Operations: Buffer

In [ ]:
# Create buffers around cities
# Buffer distance in degrees (approximately 1 degree ≈ 111 km at equator)
buffer_distance = 5  # degrees

gdf_cities['buffer'] = gdf_cities.geometry.buffer(buffer_distance)

# Visualize cities with buffers
fig, ax = plt.subplots(figsize=(12, 8))

# Plot buffers
gdf_cities.set_geometry('buffer').plot(ax=ax, 
                                        color='lightblue', 
                                        alpha=0.3, 
                                        edgecolor='blue')

# Plot city points
gdf_cities.set_geometry('geometry').plot(ax=ax, 
                                          color='red', 
                                          markersize=100)

# Add labels
for idx, row in gdf_cities.iterrows():
    ax.annotate(text=row['City'], 
                xy=(row['Longitude'], row['Latitude']),
                xytext=(5, 5), 
                textcoords='offset points',
                fontsize=9)

ax.set_title(f'Cities with {buffer_distance}° Buffer Zones')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.grid(True, alpha=0.3)
plt.show()

# Reset geometry to original
gdf_cities = gdf_cities.set_geometry('geometry')

## 9. Distance Calculations

In [ ]:
# Calculate distances between cities
from itertools import combinations

print("Distances between cities (in degrees):")
print("Note: 1 degree ≈ 111 km at the equator\n")

for (idx1, city1), (idx2, city2) in combinations(gdf_cities.iterrows(), 2):
    distance = city1.geometry.distance(city2.geometry)
    distance_km = distance * 111  # Rough approximation
    print(f"{city1['City']} <-> {city2['City']}: {distance:.2f}° (~{distance_km:.0f} km)")

## 10. Coordinate Reference Systems (CRS)

Understanding and transforming between different coordinate reference systems is crucial for accurate spatial analysis.

In [ ]:
# Check current CRS
print(f"Current CRS: {gdf_cities.crs}")

# Transform to a projected CRS (Web Mercator - EPSG:3857)
gdf_cities_projected = gdf_cities.to_crs('EPSG:3857')
print(f"\nProjected CRS: {gdf_cities_projected.crs}")

# Now calculate distance in meters using projected coordinates
moscow = gdf_cities_projected[gdf_cities_projected['City'] == 'Moscow'].geometry.iloc[0]
spb = gdf_cities_projected[gdf_cities_projected['City'] == 'Saint Petersburg'].geometry.iloc[0]

distance_meters = moscow.distance(spb)
print(f"\nDistance Moscow to Saint Petersburg: {distance_meters/1000:.2f} km")

## 11. Spatial Relationships

In [ ]:
# Create sample polygons
poly1 = Polygon([(0, 0), (2, 0), (2, 2), (0, 2)])
poly2 = Polygon([(1, 1), (3, 1), (3, 3), (1, 3)])
poly3 = Polygon([(4, 0), (6, 0), (6, 2), (4, 2)])
point_test = Point(1.5, 1.5)

# Test spatial relationships
print("Spatial Relationships:")
print(f"Poly1 intersects Poly2: {poly1.intersects(poly2)}")
print(f"Poly1 intersects Poly3: {poly1.intersects(poly3)}")
print(f"Poly1 touches Poly3: {poly1.touches(poly3)}")
print(f"Point is within Poly1: {poly1.contains(point_test)}")
print(f"Point is within Poly2: {poly2.contains(point_test)}")

# Get intersection
intersection = poly1.intersection(poly2)
print(f"\nIntersection area: {intersection.area:.2f}")

# Visualize relationships
fig, ax = plt.subplots(figsize=(10, 8))

# Plot polygons
x1, y1 = poly1.exterior.xy
ax.fill(x1, y1, alpha=0.3, fc='blue', ec='blue', linewidth=2, label='Polygon 1')

x2, y2 = poly2.exterior.xy
ax.fill(x2, y2, alpha=0.3, fc='red', ec='red', linewidth=2, label='Polygon 2')

x3, y3 = poly3.exterior.xy
ax.fill(x3, y3, alpha=0.3, fc='green', ec='green', linewidth=2, label='Polygon 3')

# Plot intersection
if not intersection.is_empty:
    x_int, y_int = intersection.exterior.xy
    ax.fill(x_int, y_int, alpha=0.7, fc='purple', ec='black', linewidth=2, label='Intersection')

# Plot point
ax.plot(point_test.x, point_test.y, 'ko', markersize=10, label='Point', zorder=5)

ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_title('Spatial Relationships')
plt.show()

## 12. Spatial Join

Spatial joins combine datasets based on spatial relationships rather than common attributes.

In [ ]:
# Find which country each city is in
cities_with_countries = gpd.sjoin(gdf_cities, world, how='left', predicate='within')

print("Cities with country information:")
print(cities_with_countries[['City', 'Population', 'name', 'continent', 'pop_est']])

## 13. Area and Boundary Analysis

In [ ]:
# Calculate areas of countries (in geographic coordinates - not accurate)
world['area_deg'] = world.geometry.area

# For accurate area calculation, convert to an equal-area projection
# Using World Mollweide (EPSG:54009)
world_projected = world.to_crs('ESRI:54009')
world_projected['area_km2'] = world_projected.geometry.area / 1e6  # Convert to km²

# Top 10 largest countries by area
largest_countries = world_projected.nlargest(10, 'area_km2')[['name', 'area_km2']]
print("Top 10 Largest Countries by Area:")
print(largest_countries.to_string(index=False))

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))
largest_countries.plot(x='name', y='area_km2', kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Area (km²)')
ax.set_title('Top 10 Largest Countries')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 14. Centroid Calculation

In [ ]:
# Calculate centroids of European countries
europe['centroid'] = europe.geometry.centroid

# Visualize countries and their centroids
fig, ax = plt.subplots(figsize=(12, 10))

europe.plot(ax=ax, color='lightblue', edgecolor='black', linewidth=0.5)
europe.set_geometry('centroid').plot(ax=ax, color='red', markersize=30, label='Centroids')

ax.set_title('European Countries with Centroids')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

# Reset geometry
europe = europe.set_geometry('geometry')

## 15. Interactive Maps with Folium

In [ ]:
# Create an interactive map centered on Russia
m = folium.Map(location=[60, 50], zoom_start=3)

# Add city markers
for idx, row in gdf_cities.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Population']/500000,
        popup=f"{row['City']}<br>Population: {row['Population']:,}",
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.6
    ).add_to(m)

# Display map
m

## 16. Heatmap Visualization

In [ ]:
# Create a heatmap of city populations
m = folium.Map(location=[60, 50], zoom_start=3)

# Prepare data for heatmap
heat_data = [[row['Latitude'], row['Longitude'], row['Population']/10000] 
             for idx, row in gdf_cities.iterrows()]

# Add heatmap layer
plugins.HeatMap(heat_data, radius=50).add_to(m)

# Display map
m

## 17. Spatial Clustering with K-means

Let's create synthetic spatial data and perform clustering.

In [ ]:
from sklearn.cluster import KMeans

# Generate random spatial points
np.random.seed(42)
n_points = 200

# Create clusters of points
cluster1 = np.random.randn(n_points//4, 2) * 0.5 + [0, 0]
cluster2 = np.random.randn(n_points//4, 2) * 0.5 + [3, 3]
cluster3 = np.random.randn(n_points//4, 2) * 0.5 + [0, 3]
cluster4 = np.random.randn(n_points//4, 2) * 0.5 + [3, 0]

points = np.vstack([cluster1, cluster2, cluster3, cluster4])

# Perform K-means clustering
kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(points)
centroids = kmeans.cluster_centers_

# Visualize results
plt.figure(figsize=(12, 10))
scatter = plt.scatter(points[:, 0], points[:, 1], c=labels, cmap='viridis', alpha=0.6, s=50)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200, 
            edgecolors='black', linewidths=2, label='Centroids')
plt.colorbar(scatter, label='Cluster')
plt.title('Spatial K-means Clustering')
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

## 18. DBSCAN Clustering for Spatial Data

In [ ]:
from sklearn.cluster import DBSCAN

# Apply DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
labels_dbscan = dbscan.fit_predict(points)

# Visualize results
plt.figure(figsize=(12, 10))
scatter = plt.scatter(points[:, 0], points[:, 1], c=labels_dbscan, 
                     cmap='viridis', alpha=0.6, s=50)
plt.colorbar(scatter, label='Cluster')
plt.title('Spatial DBSCAN Clustering')
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

print(f"Number of clusters: {len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)}")
print(f"Number of noise points: {list(labels_dbscan).count(-1)}")

## 19. Convex Hull

In [ ]:
# Create convex hulls for each cluster
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# K-means clusters with convex hulls
axes[0].scatter(points[:, 0], points[:, 1], c=labels, cmap='viridis', alpha=0.6, s=50)
for i in range(4):
    cluster_points = points[labels == i]
    if len(cluster_points) > 2:
        hull = MultiPoint(cluster_points.tolist()).convex_hull
        if hull.geom_type == 'Polygon':
            x, y = hull.exterior.xy
            axes[0].plot(x, y, 'r-', linewidth=2, alpha=0.7)
            axes[0].fill(x, y, alpha=0.2)

axes[0].set_title('K-means Clusters with Convex Hulls')
axes[0].set_xlabel('X coordinate')
axes[0].set_ylabel('Y coordinate')
axes[0].grid(True, alpha=0.3)
axes[0].axis('equal')

# DBSCAN clusters with convex hulls
axes[1].scatter(points[:, 0], points[:, 1], c=labels_dbscan, cmap='viridis', alpha=0.6, s=50)
for i in set(labels_dbscan):
    if i != -1:  # Skip noise
        cluster_points = points[labels_dbscan == i]
        if len(cluster_points) > 2:
            hull = MultiPoint(cluster_points.tolist()).convex_hull
            if hull.geom_type == 'Polygon':
                x, y = hull.exterior.xy
                axes[1].plot(x, y, 'r-', linewidth=2, alpha=0.7)
                axes[1].fill(x, y, alpha=0.2)

axes[1].set_title('DBSCAN Clusters with Convex Hulls')
axes[1].set_xlabel('X coordinate')
axes[1].set_ylabel('Y coordinate')
axes[1].grid(True, alpha=0.3)
axes[1].axis('equal')

plt.tight_layout()
plt.show()

## 20. Nearest Neighbor Analysis

In [ ]:
from scipy.spatial import cKDTree

# Find k nearest neighbors for each city
coords = np.array([[row['Longitude'], row['Latitude']] for idx, row in gdf_cities.iterrows()])
tree = cKDTree(coords)

# Find 2 nearest neighbors for each city (including itself)
k = 2
distances, indices = tree.query(coords, k=k)

print("Nearest neighbor for each city:")
for i, (idx, row) in enumerate(gdf_cities.iterrows()):
    nearest_idx = indices[i, 1]  # Index 0 is the point itself
    nearest_city = gdf_cities.iloc[nearest_idx]['City']
    dist = distances[i, 1]
    dist_km = dist * 111  # Rough approximation
    print(f"{row['City']}: nearest is {nearest_city} (~{dist_km:.0f} km away)")

## 21. Exercise: Your Own Spatial Analysis

Create your own spatial dataset and perform the following analyses:

1. Create a GeoDataFrame with at least 10 points of interest
2. Calculate distances between points
3. Create buffer zones around points
4. Perform spatial clustering
5. Create an interactive map visualization

In [ ]:
# Your code here
# Example: Create a dataset of landmarks, restaurants, or any spatial data you're interested in

# my_data = {
#     'name': [...],
#     'lat': [...],
#     'lon': [...],
# }
# Perform analysis

## Summary

In this notebook, we covered:
- Creating and visualizing geometric objects (points, lines, polygons)
- Working with GeoDataFrames
- Coordinate Reference Systems and transformations
- Spatial operations (buffers, intersections, unions)
- Distance calculations and nearest neighbor analysis
- Spatial joins and relationships
- Area and boundary analysis
- Interactive map visualization with Folium
- Spatial clustering (K-means, DBSCAN)
- Convex hull computation

## Further Resources

- GeoPandas documentation: https://geopandas.org/
- Shapely documentation: https://shapely.readthedocs.io/
- Folium documentation: https://python-visualization.github.io/folium/
- "Python for Geospatial Data Analysis" by Bonny P. McClain
- "Geographic Data Science with Python" (online book)